In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
import os
import warnings
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

from quantnn.qrnn import QRNN
from quantnn.models.pytorch.logging import TensorBoardLogger

import sys
sys.path.append('../visualize')
from sample_plots import plotRandomSample
sys.path.append('../src')
from load_data import GOESRETRIEVALSDataset, RandomLog, Mask, RandomCrop, Standardize, ToTensor
from models.FirstGenericNet import Net 
net_name = 'FirstGenericNet' 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

In [ ]:
# SETUP
channels = list(range(8,17))
channels.remove(12)

fillvalue = -1

n_epochs = 20
BATCH_SIZE = 16

quantiles = [0.25, 0.5, 0.75] #[0.01, 0.05, 0.15, 0.25, 0.35, 0.45, 0.5, 0.55, 0.65, 0.75, 0.85, 0.95, 0.99]

In [ ]:
# PATHS TO DATA
path_to_data = '../dataset/data/dataset-test/'
path_to_save_model = '../results/saved_models/'

In [ ]:
path_to_train_data = path_to_data + 'train/npy_files'
path_to_stats = os.path.join(Path(path_to_train_data).parent, Path('stats.npy'))

path_to_val_data = path_to_data + 'validation/npy_files'

path_to_test_data = path_to_data + 'test/npy_files'

In [ ]:
def importData(channels, BATCH_SIZE, path_to_data, path_to_stats):
    dataset = GOESRETRIEVALSDataset(
        path_to_data = path_to_data,
        channels = channels, 
        transform = transforms.Compose([
            #RandomLog(),
            Mask(), 
            RandomCrop(128),
            Standardize(path_to_data, path_to_stats, channels),
            ToTensor()])
    )

    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    return(dataset, dataloader)

In [ ]:
train_dataset, train_dataloader = importData(channels, BATCH_SIZE, path_to_train_data, path_to_stats)

In [ ]:
val_dataset, val_dataloader = importData(channels, BATCH_SIZE, path_to_val_data, path_to_stats)

In [ ]:
# PLOT RANDOM SAMPLE
plotRandomSample(train_dataset)

In [ ]:
plotRandomSample(val_dataset, c=[2,7])

In [ ]:
net = Net(len(quantiles), len(channels))
qrnn_model = QRNN(quantiles=quantiles, model=net)
optimizer = SGD(net.parameters(), lr=0.1, momentum=0.9)

In [ ]:
training_data = train_dataloader
validation_data = val_dataloader

In [ ]:
log_directory = None
logger = TensorBoardLogger(n_epochs, log_directory=log_directory)

In [ ]:
scheduler = CosineAnnealingLR(optimizer, n_epochs, 0.001)
qrnn_model.train(training_data=training_data,
              validation_data=validation_data,
              keys=("box", "label"),
              n_epochs=n_epochs,
              optimizer=optimizer,
              scheduler=scheduler,
              mask=fillvalue,
              device=device,
              logger=logger);
scheduler = CosineAnnealingLR(optimizer, n_epochs, 0.0001)
qrnn_model.train(training_data=training_data,
              validation_data=validation_data,
              keys=("box", "label"),
              n_epochs=n_epochs,
              optimizer=optimizer,
              scheduler=scheduler,
              mask=fillvalue,
              device=device,
              logger=logger);

In [ ]:
# SAVE MODEL
#qrnn_model.save(path_to_save_model+'qrnn_model')

In [ ]:
plotRandomSample(val_dataset, qrnn=qrnn_model, device=device)

In [ ]:
quantile_num=int(len(quantiles)/2)
y_true = []
y_pred = []
with torch.no_grad():
    for batch_index, batch in enumerate(val_dataloader):

        y_true += [batch['label'].detach().numpy()]
        
        X = batch['box'].to(device).detach()
        y_pred += [net(X)[:,quantile_num].cpu().detach().numpy()]

y_true = np.concatenate(y_true, axis=0)
y_pred = np.concatenate(y_pred, axis=0)

In [ ]:
bins = np.logspace(-2, 2, 81)
indices = y_true >= 0.0


freqs, _, _ = np.histogram2d(y_true[indices], y_pred[indices], bins=bins)

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.colors import Normalize
#norm = Normalize(0, 400)

f, axs = plt.subplots(figsize=(5, 6))

ax = axs
p = ax.pcolormesh(bins, bins, freqs.T,
                  #norm=norm,  
                  cmap=plt.get_cmap('magma'))
ax.set_xlim([1e-2, 1e2])
ax.set_ylim([1e-2, 1e2])
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Reference rain rate [mm / h]")
ax.set_ylabel("Predicted rain rate [mm / h]")
#ax.set_title("(a) Title", loc="left")
ax.plot(bins, bins, c="grey", ls="--")
f.colorbar(p, ax=ax, orientation="horizontal", label="Surface precipitation [mm / h]")
ax.set_aspect(1.0)


In [ ]:
print('Non zero rain to all rain ratio (true): ', np.sum(y_true[indices]!=0.)/len(y_true[indices]))
print('Non zero rain to all rain ratio (pred): ', np.sum(y_pred[indices]!=0.)/len(y_pred[indices]))

TP = np.sum(y_pred[indices][(y_true[indices]!=0.)]!=0.)
FN = np.sum(y_pred[indices][(y_true[indices]!=0.)]==0.)
FP = np.sum(y_pred[indices][(y_true[indices]==0.)]!=0.)
TN = np.sum(y_pred[indices][(y_true[indices]==0.)]==0.)

TPR = TP/(TP+FN)
print(TPR)
FPR = FP/(FP+TN)
print(FPR)

ACC = (TP+TN)/(TP+FN+FP+TN)
print(ACC)

In [ ]:
print(len(y_true[indices]))
print(len(val_dataset))


In [ ]:
(len(train_dataset))